In [1]:
import pathlib
import pandas as pd
import geopandas as gpd
import os
from bs4 import BeautifulSoup

os.environ["OGR_GEOMETRY_ACCEPT_UNCLOSED_RING"] = "NO"


def extract_name(description):
    soup = BeautifulSoup(description)
    rows = soup.find_all("tr")
    th_td = dict()
    for row in rows:
        th = row.find("th")
        td = row.find("td")
        if th and td:
            th_td[th.text.strip()] = td.text.strip()

    if th_td.get("RAIL_TYPE", None) in ("LRT", "MRT") and th_td.get("NAME", None):
        return th_td["NAME"].removesuffix(" INTERCHANGE")
    return None


def get_all_coordinates_mp2019():
    # Source: https://data.gov.sg/datasets/d_9a6bdc9d93bd041eb0cfbb6a8cb3248f/view
    gdf = gpd.read_file(
        pathlib.Path("data") / "AmendmenttoMasterPlan2019RailStationlayer.geojson"
    ).to_crs(epsg=4326)
    gdf["Name"] = gdf["Description"].apply(extract_name)
    gdf.crs = None  # Suppress warnings.
    gdf["centroid"] = gdf.geometry.centroid
    gdf["lat"] = gdf["centroid"].y
    gdf["lon"] = gdf["centroid"].x
    return gdf.dropna(axis=0)


gdf = get_all_coordinates_mp2019()

# Replace openstreetmap coordinates with URA coordinates
# URA data does not have station codes to disambiguate interchange stations. Let's handle non-interchanges first.
gdf_without_interchanges = gdf.drop_duplicates(subset="Name", keep=False)
original_df = pd.read_csv("future_stations.csv", header=0)
df = original_df.copy(deep=True)
df["station_name_upper"] = df["station_name"].str.upper()
df = df[df["source"] == "openstreetmap"].merge(
    gdf_without_interchanges.drop(columns=["Description", "geometry", "centroid"]),
    how="left",
    left_on="station_name_upper",
    right_on="Name",
)
df["lat_x"] = df["lat_y"]
df["lon_x"] = df["lon_y"]
df["source"] = "ura"
df = df.drop(columns=["station_name_upper", "Name", "lat_y", "lon_y"])
df = df.dropna(subset=["lat_x", "lon_x"])
merged_df = original_df.merge(df, on="station_code", how="left")
original_df["lat"] = merged_df["lat_x"].combine_first(merged_df["lat"])
original_df["lon"] = merged_df["lon_x"].combine_first(merged_df["lon"])
original_df["source"] = merged_df["source_y"].fillna(merged_df["source_x"])
print(original_df.to_csv(index=False))

station_code,station_name,lat,lon,source,comment
NE18,Punggol Coast,1.41492733388605,103.910166388177,onemap,nel_pe
PW2,Teck Lee,1.4127708937463,103.906577622768,onemap,
DT4,Hume,1.354510835593032,103.76910392895525,ura,
TE30,Bedok South,1.31668400658419,103.94931120983,onemap,tel_5
TE31,Sungei Bedok,1.32040148729112,103.957183823478,onemap,tel_5
DT36,Xilin,1.32890694938858,103.964902569595,onemap,dtl_3e
DT37,Sungei Bedok,1.32040148729112,103.957183823478,onemap,dtl_3e
CC30,Keppel,1.26977220124441,103.830031459524,onemap,ccl_6
CC31,Cantonment,1.27287214563203,103.837062313764,onemap,ccl_6
CC32,Prince Edward Road,1.27315693636085,103.847097123745,onemap,ccl_6
JS1,Choa Chu Kang,1.385382350507513,103.74411408003186,ura,jrl_1
JS2,Choa Chu Kang West,1.379221009084314,103.7400134290408,ura,jrl_1
JS3,Tengah,1.3663843951975398,103.73008354165452,ura,jrl_1
JS4,Hong Kah,1.35812200829127,103.726153007988,onemap,jrl_1
JS5,Corporation,1.352969534325842,103.7138129954562,ura,jrl_1
JS6,Jurong West,1.

In [2]:
# Find future interchange stations. Most of them are in JRL or CRL.
gdf_interchanges_only = gdf[gdf["Name"].duplicated(keep=False)]
for _, row in (
    gdf_interchanges_only[
        gdf_interchanges_only["Name"].isin(original_df["station_name"].str.upper())
    ]
    .sort_values(by=["Name", "centroid"])
    .iterrows()
):
    print(f'{row["Name"]},{row["lat"]},{row["lon"]}')

# Now manually edit future_stations.csv as there are no station codes to disambiguate stations within interchange.
# Afterwards, cross-reference with online map services.

ANG MO KIO,1.3699486592675338,103.84961635287675
ANG MO KIO,1.3693883850961412,103.85090944829145
BOON LAY,1.3381080254056505,103.70457367154302
BOON LAY,1.3386044527608785,103.7060661291329
CHOA CHU KANG,1.3848220120699979,103.7445491163712
CHOA CHU KANG,1.3853823505075127,103.74411408003186
CHOA CHU KANG,1.3851720417705848,103.74436907015479
HOUGANG,1.3709756026538165,103.89114720923538
HOUGANG,1.3714546592579868,103.89235006980195
JURONG EAST,1.3329658528821977,103.74190976562035
JURONG EAST,1.333134716477479,103.74230519802595
PASIR RIS,1.3731161767548992,103.94929460955761
PASIR RIS,1.371259477431133,103.94888291887848
